<a href="https://colab.research.google.com/github/lumasg/Chatbot/blob/main/Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Configurar ambiente

In [1]:
#Instalando bibliotecas
import pandas as pd
import re, os, random, pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import spacy
!python -m spacy download es_core_news_md
!pip install jellyfish
import jellyfish
!pip install transformers
from transformers import BertForSequenceClassification
from transformers import BertTokenizer
import torch
!pip install python-docx
from docx import Document
import csv
import nltk
nltk.download('punkt')

#Definiendo variables del proyecto:
nlp = spacy.load('es_core_news_md')

#Conectando al Google Drive
from google.colab import drive
drive.mount('drive')
folder = 'drive/MyDrive/Data_Sciencie/Chatbot'

2023-10-03 21:04:30.247539: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 21.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_md')
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 59.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 92.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 37.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for python-docx: filename=python_docx-0.8.11-py3-none-any.whl size=184487 sha25

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Mounted at drive


# 2. Importar verbos

In [2]:
lista_verbos = "drive/MyDrive/Data_Sciencie/Chatbot/verbos/lista_verbos.pickle"
lista_verbos_irregulares = "drive/MyDrive/Data_Sciencie/Chatbot/verbos/verbos_irregulares.pickle"

In [3]:
# Importar la lista_verbos:
with open(lista_verbos, 'rb') as verbos:
  lista_verbos = pickle.load(verbos)

# Importar el diccionario:
with open(lista_verbos_irregulares, 'rb') as verbosi:
  lista_verbos_irregulares = pickle.load(verbosi)

In [4]:
print(lista_verbos)
print("\n")
print(lista_verbos_irregulares)

['parar', 'recomendar', 'cancelar', 'fanatizar', 'amaran o amasen', 'exponer', 'obedecer', 'quejar', 'echar', 'legitimar', 'perjudicar', 'organizar', 'molar', 'objetar', 'considerar', 'golear', 'mover', 'acertar', 'reunir', 'regir', 'ilusionar', 'simpatizar', 'conjeturar', 'helar', 'quitar', 'amariamos', 'destacar', 'llegar', 'sincronizar', 'lesionar', 'seducir', 'asistir', 'conservar', 'acordar', 'salvar', 'relucir', 'graduar', 'forzar', 'dar', 'deplorar', 'batear', 'mofar', 'estropear', 'aplastar', 'wasapeo', 'gestionar', 'suprimir', 'gruñir', 'progresar', 'suscribir', 'noticiar', 'cavar', 'alejar', 'galopar', 'virar', 'medir', 'actualizar', 'humanizar', 'convivir', 'gratificar', 'digerir', 'tocar', 'zonificar', 'amariais', 'aterrizar', 'hojear', 'cometer', 'sufrir', 'reciclar', 'obturar', 'divertir', 'ondear', 'listar', 'determinar', 'alentar', 'sumar', 'reflexionar', 'ames', 'anotar', 'mitificar', 'escribir', 'ilustrar', 'obtener', 'subir', 'socorrer', 'desprender', 'agregar', 'gan

#3. Tratamiento de datos

In [5]:
#Función para encontrar la raiz de las palabras
def raiz(palabra):
    max_similitud = 0.0
    for verbo in lista_verbos:
        similitud = jellyfish.jaro_similarity(palabra, verbo)
        if similitud > max_similitud:
            max_similitud = similitud
            palabra_encontrada = verbo

    if max_similitud >= 0.93:
        return palabra_encontrada
    else:
        return palabra

def tratamiento_texto(texto):
  trans = str.maketrans('áéíóúâà','aeiouaa')
  texto = texto.lower()
  texto = texto.translate(trans)
  texto = re.sub(r"[^\w\s+\-*/]", '', texto)
  texto = " ".join(texto.split())
  # print('texto de tratamiento_texto: ',texto)
  return texto

#Función para reemplazar el final de una palabra por 'r'
def reemplazar_terminacion(frase):
    terminaciones = ["es", "me", "as", "ste", "te"]
    palabras = frase.split()

    for i, palabra in enumerate(palabras):
        for terminacion in terminaciones:
            if palabra.endswith(terminacion) and len(palabra) > len(terminacion):
                palabras[i] = palabra[:-len(terminacion)] + "r"
                break

    nueva_frase = " ".join(palabras)
    return nueva_frase

#Función para adicionar o eliminar tokens
def revisar_tokens(texto, tokens):
  texto=tratamiento_texto(texto)
  if len(tokens)==0:
    if any(name in texto for name in ['cientifico de datos', 'data scientist']):
        tokens.append('datascientist')
    if any(name in texto for name in ['elprofealejo', 'el profe alejo', 'profe alejo', 'profealejo']):
        tokens.append('elprofealejo')
    if any(name in texto for name in ['ciencia de datos', 'data science']):
        tokens.append('datascience')
  else:
    elementos_a_eliminar = ["cual", "que", "quien", "cuanto", "cuando", "como"]
    if 'hablame' in texto and 'hablar' in tokens: elementos_a_eliminar.append('hablar')
    elif 'cuentame' in texto and 'contar' in tokens: elementos_a_eliminar.append('contar')
    elif 'hago' in texto and 'hacer' in tokens: elementos_a_eliminar.append('hacer')
    elif 'entiendes' in texto and 'entender' in tokens: elementos_a_eliminar.append('entender')
    elif 'sabes' in texto and 'saber' in tokens: elementos_a_eliminar.append('saber')
    tokens = [token for token in tokens if token not in elementos_a_eliminar]
  return tokens


#Función para devolver los tokens normalizados del texto
def normalizar(texto):
    tokens = []
    tokens = revisar_tokens(texto, tokens)
    doc = nlp(texto)
    for t in doc:
        # Obtener el lemma
        lemma = lista_verbos_irregulares.get(t.text, t.lemma_)

        # Verificar si lemma es una cadena no vacía
        if lemma and isinstance(lemma, str):
            lemma = re.sub(r'[^\w\s+\-*/]', '', lemma)

            if t.pos_ in ('VERB','PROPN','PRON','NOUN','AUX','SCONJ','ADJ','ADV','NUM') or lemma in lista_verbos:
                if t.pos_ == 'VERB':
                    lemma = reemplazar_terminacion(lemma)
                    tokens.append(raiz(tratamiento_texto(lemma)))
                else:
                    tokens.append(tratamiento_texto(lemma))

    tokens = list(dict.fromkeys(tokens))
    tokens = list(filter(None, tokens))
    tokens = revisar_tokens(texto, tokens)
    tokens_str = str(tokens)
    # print('tokens: ',tokens_str)
    return tokens_str

In [6]:
normalizar("profe alejo cientifico de datos")

"['datascientist', 'elprofealejo', 'profe', 'alejo', 'cientifico', 'dato']"

# 4. Cargar bases de documentos

In [8]:
#Importando bases de dialogo fluído
txt_folder_path = folder + '/dialogos'
lista_documentos=[x for x in os.listdir(txt_folder_path) if x.endswith(".txt")]
lista_dialogos, lista_dialogos_respuesta, lista_tipo_dialogo = [],[],[]
for idx in range(len(lista_documentos)):
  f=open(txt_folder_path+'/'+lista_documentos[idx], 'r', encoding='utf-8', errors='ignore')
  estado = True
  for line in f.read().split('\n'):
    if estado:
      line_tratado = tratamiento_texto(line)
      lista_dialogos.append(line_tratado)
      lista_tipo_dialogo.append(lista_documentos[idx][:-4])
    else:
      lista_dialogos_respuesta.append(line)
    estado=not estado

#Creando Dataframe de diálogos
datos = {'dialogo':lista_dialogos,'respuesta':lista_dialogos_respuesta,'tipo':lista_tipo_dialogo,'interseccion':0,'jaro_winkler':0,'probabilidad':0}
df_dialogo = pd.DataFrame(datos)
df_dialogo = df_dialogo.drop_duplicates(keep='first')
df_dialogo.reset_index(drop=True, inplace=True)

#Importando bases csv
txt_folder_path = folder+'/documentos'
lista_documentos=[x for x in os.listdir(txt_folder_path) if x.endswith(".csv")]
documento_csv = ''
for i in range(len(lista_documentos)):
  with open(txt_folder_path+'/'+lista_documentos[i], "r", encoding="utf-8") as csv_txt:
    csv_text = csv.reader(csv_txt)
    for fila in csv_text:
      if fila[-1]!='frase':
        documento_csv += fila[-1]

#Importando bases docx
lista_documentos=[x for x in os.listdir(txt_folder_path) if x.endswith(".docx")]
documento_docx = ''
for i in range(len(lista_documentos)):
  for t in Document(txt_folder_path+'/'+lista_documentos[i]).paragraphs:
    documento_docx += t.text.replace('*','\n\n*')+ "\n"
    # documento_docx += t.text.replace('*','\n\n*').replace('-','\n-')
    # documento_docx += documento_docx + "\n"

#Importando bases txt
lista_documentos=[x for x in os.listdir(txt_folder_path) if x.endswith(".txt")]
documento_txt = ''
for i in range(len(lista_documentos)):
  with open(txt_folder_path+'/'+lista_documentos[i], "r", encoding="utf-8") as txt:
    txt_new = txt.read()
    for i in txt_new:
      documento_txt += i

In [9]:
#Creando Dataframe de diálogos
datos = {'dialogo':lista_dialogos,'respuesta':lista_dialogos_respuesta,'tipo':lista_tipo_dialogo,'interseccion':0,'jaro_winkler':0,'probabilidad':0}
df_dialogo = pd.DataFrame(datos)
df_dialogo = df_dialogo.drop_duplicates(keep='first')
df_dialogo.reset_index(drop=True, inplace=True)

In [10]:
df_dialogo

,dialogo,respuesta,tipo,interseccion,jaro_winkler,probabilidad
0,y cuantos años tienes,Tengo menos de 1 año.,Edad,0,0,0
1,cuantos años tienes,Tengo menos de 1 año.,Edad,0,0,0
2,que edad tenes,Actualmente tengo menos de 1 año.,Edad,0,0,0
3,que edad tienes,Actualmente tengo menos de 1 año.,Edad,0,0,0
4,cuanto tiempo has estado activo,He estado activo como chatbot desde mi fecha d...,Edad,0,0,0
...,...,...,...,...,...,...
1147,prueba api whatsapp,¿En qué puedo ayudarte hoy humano?,Saludos,0,0,0
1148,prueba de chat,Hola! ¿En qué puedo ayudarte?,Saludos,0,0,0
1149,prueba de chat gpt,"No, no soy chat GPT. ¡Soy ibot el chatbot más ...",Saludos,0,0,0
1150,solo te saludd,Me alegra saludarte. Espero que estés bien.,Saludos,0,0,0


# 5. Buscar respuesta del Chatbot

In [11]:
def guardar_dataframe(dataframe):
  carpeta = "drive/MyDrive/Data_Sciencie/Chatbot"
  nombre_archivo = "df_dialogo.csv"

  if not os.path.exists(carpeta):
    os.makedirs(carpeta)
  archivo_csv = os.path.join(carpeta, nombre_archivo)
  dataframe.to_csv(archivo_csv, index=False)

In [12]:
def dialogo(user_response):
    # Tratamiento de texto
    user_response = tratamiento_texto(user_response)  # Tratando el texto
    user_response = re.sub(r"[^\w\s]", '', user_response)  # Elimina signos de puntuación

    df = df_dialogo.copy()

    vectorizer = TfidfVectorizer()

    for idx, row in df.iterrows():

        dialogos_numero = vectorizer.fit_transform(df_dialogo['dialogo'])
        respuesta_numero = vectorizer.transform([user_response])
        cos_sim = cosine_similarity(dialogos_numero[idx], respuesta_numero)[0][0]

        df.at[idx, 'interseccion'] = len(set(user_response.split()) & set(row['dialogo'].split()))/len(user_response.split())
        df.at[idx, 'similarity'] = cos_sim
        df.at[idx, 'jaro_winkler'] = jellyfish.jaro_winkler_similarity(user_response, row['dialogo'])

        # Calcular la probabilidad:
        df.at[idx, 'probabilidad'] = max(df.at[idx, 'interseccion'], df.at[idx, 'similarity'], df.at[idx, 'jaro_winkler'])

    df.sort_values(by=['probabilidad', 'jaro_winkler'], inplace=True, ascending=False)

    # Probabilidad máxima:
    probabilidad = round(df['probabilidad'].head(1).values[0],2)

    #Guardar df:
    guardar_dataframe(df)

    if probabilidad >= 0.93:
        print('Respuesta encontrada por el método de comparación de textos - Probabilidad: ', probabilidad)
        respuesta = df['respuesta'].head(1).values[0]
    else:
        respuesta = ''
    return respuesta

#Cargar tu modelo entrenado aqui(recuerda siempre cargar el modelo y el vectorizer o tokenizer usado en el entrenamiento del modelo):
ruta_modelo = 'drive/MyDrive/Data_Sciencie/Chatbot/modelo'
Modelo_TF = BertForSequenceClassification.from_pretrained(ruta_modelo)
tokenizer_TF = BertTokenizer.from_pretrained(ruta_modelo)

#Función para dialogar utilizando el modelo de Machine Learning:
def clasificacion_modelo(pregunta):
  frase = normalizar(pregunta)
  frase = ' '.join(str(elemento) for elemento in frase)
  tokens = tokenizer_TF.encode_plus(
      frase,
      add_special_tokens=True,
      max_length=128,
      padding='max_length',
      truncation=True,
      return_tensors='pt'
  )
  input_ids = tokens['input_ids']
  attention_mask = tokens['attention_mask']

  with torch.no_grad():
      outputs = Modelo_TF(input_ids, attention_mask)

  etiquetas_predichas = torch.argmax(outputs.logits, dim=1)
  etiquetas_decodificadas = etiquetas_predichas.tolist()

  diccionario = {3: 'Continuacion', 10: 'Nombre', 2: 'Contacto', 13: 'Saludos', 14: 'Sentimiento', 9: 'Identidad', 15: 'Usuario', 6: 'ElProfeAlejo', 1: 'Aprendizaje', 0: 'Agradecimiento', 5: 'Edad', 4: 'Despedida', 11: 'Origen', 12: 'Otros', 7: 'Error', 8: 'Funcion'}
  llave_buscada = etiquetas_decodificadas[0]
  clase_encontrada = diccionario[llave_buscada]

  #Buscar respuesta más parecida en la clase encontrada
  df = df_dialogo[df_dialogo['tipo'] == clase_encontrada]
  df.reset_index(inplace=True)
  vectorizer = TfidfVectorizer()
  dialogos_num = vectorizer.fit_transform(df['dialogo'])
  pregunta_num = vectorizer.transform([tratamiento_texto(pregunta)])
  similarity_scores = cosine_similarity(dialogos_num, pregunta_num)
  indice_pregunta_proxima = similarity_scores.argmax()

  if max(similarity_scores)>0.5 and clase_encontrada not in ['Otros']:
    print('Respuesta encontrada por el modelo Transformers - tipo:',clase_encontrada)
    respuesta = df['respuesta'][indice_pregunta_proxima]
  else:
    respuesta = ''
  return respuesta

# Función para devolver la respuesta de los documentos
def respuesta_documento(pregunta):
  pregunta = normalizar(pregunta)
  def contar_coincidencias(frase):
    return sum(1 for elemento in pregunta if elemento in frase)

  diccionario = {valor: posicion for posicion, valor in enumerate(lista_frases_normalizadas)}
  lista = sorted(list(diccionario.keys()), key=contar_coincidencias, reverse=True)[:100]

  lista.append(''.join(pregunta))

  TfidfVec = TfidfVectorizer(tokenizer=normalizar)
  tfidf = TfidfVec.fit_transform(lista)

  vals = cosine_similarity(tfidf[-1], tfidf)
  idx = vals.argsort()[0][-2]
  flat = vals.flatten()
  flat.sort()
  req_tfidf = round(flat[-2],2)
  if req_tfidf>=0.22:
    print('Respuesta encontrada por el método TfidfVectorizer - Probabilidad:', req_tfidf)
    respuesta = lista_frases[diccionario[lista[idx]]]
  else:
    respuesta = ''
  return respuesta

# Función para devolver una respuesta final buscada en todos los métodos disponibles
def respuesta_chatbot(pregunta):
  respuesta = dialogo(pregunta)
  if respuesta != '':
    return respuesta
  else:
    respuesta = clasificacion_modelo(pregunta)
    if respuesta != '':
      return respuesta
    else:
      preg=pregunta
      respuesta = respuesta_documento(preg)
      if respuesta != '':
        return respuesta
      else:
        return 'Respuesta no encontrada'

In [14]:
dialogo("como te llamas")

Respuesta encontrada por el método de comparación de textos - Probabilidad:  1.0


'Me llamo ibot. ¿Y tú?'

In [ ]:
def cargar_dataframe():
    carpeta = "drive/MyDrive/Data_Sciencie/Chatbot"
    nombre_archivo = "df_dialogo.csv"
    archivo_csv = os.path.join(carpeta, nombre_archivo)

    if os.path.exists(archivo_csv):
        dataframe = pd.read_csv(archivo_csv)
        return dataframe
    else:
        print(f"El archivo {archivo_csv} no existe.")
        return None


In [ ]:
cargar_dataframe()

,dialogo,respuesta,tipo,interseccion,jaro_winkler,probabilidad,similarity
0,me interesa curso de programacion,Debes revisar el canal de ElProfeAlejo en Youtube,Otros,1,0.583838,1.000000,0.590635
1,que servicios tienes,Mi función es responder tus dudas sobre Cienci...,Funcion,0,0.683333,0.683333,0.000000
2,escribe un modelo de,"No estoy programado para ayudarte con esto, só...",Otros,0,0.683333,0.683333,0.000000
3,cual es tu origen,"Fui desarrollado por ElProfeAlejo, utilizando ...",Origen,0,0.678431,0.678431,0.000000
4,que sos,"Me llamo Madre, ¡el chatbot más guay del barri...",Identidad,0,0.676190,0.676190,0.000000
...,...,...,...,...,...,...,...
1147,mi ip,Tu dirección IP es la dirección que tu disposi...,Otros,0,0.000000,0.000000,0.000000
1148,mmm,"Si tienes más dudas, pregúntame de nuevo!",Otros,0,0.000000,0.000000,0.000000
1149,no tu,"No, prefiero no responder a esa pregunta.",Otros,0,0.000000,0.000000,0.000000
1150,vete pal diablo,¡No digas eso! ¡Sé más respetuoso humano!,Otros,0,0.000000,0.000000,0.000000


# 6. Ejecutar Chatbot

In [15]:
pregunta ='CÓMO ESTAS'
respuesta = respuesta_chatbot(pregunta)
print(respuesta)

Respuesta encontrada por el método de comparación de textos - Probabilidad:  1.0
Estoy bien, gracias por preguntar.


In [16]:
pregunta ='CÓMO te   lamas'
respuesta = respuesta_chatbot(pregunta)
print(respuesta)

Respuesta encontrada por el método de comparación de textos - Probabilidad:  0.99
Me llamo ibot. ¿Y tú?
